In [1]:
import os
# os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
# os.environ['CUDA_VISIBLE_DEVICES'] = "7"
# os.environ["MKL_NUM_THREADS"] = "4" 
# os.environ["NUMEXPR_NUM_THREADS"] = "4" 
# os.environ["OMP_NUM_THREADS"] = "4" 

import argparse
from torch.utils.data import Dataset
from torchvision import transforms
import torch
import torch.nn as nn
import numpy as np
import math
from models import CVM_VIGOR as CVM
from datasets import VIGORDataset
import PIL.Image
from PIL import ImageFile
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from training_utils import *

%load_ext autoreload
%autoreload 2

In [2]:
# parser = argparse.ArgumentParser()
# parser.add_argument('--area', type=str, help='samearea or crossarea', default='samearea')
# parser.add_argument('--pos_only', choices=('True','False'), default='True')
# parser.add_argument('--ori_noise', type=float, help='noise in orientation prior, 180 means unknown orientation', default=180.)
# parser.add_argument('--idx', type=int, help='image index')

# args = vars(parser.parse_args())
area = 'samearea'
ori_noise = 180.
ori_noise = 18 * (ori_noise // 18) # round the closest multiple of 18 degrees within prior 
pos_only = True
use_osm = True
use_adapt = False
training = False
use_concat=False

dataset_root='/scratch/izar/qngo/VIGOR'

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(17)
np.random.seed(0)

transform_grd = transforms.Compose([
    transforms.Resize([320, 640]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    
])

transform_sat = transforms.Compose([
    # resize
    transforms.Resize([512, 512]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    
])

vigor = VIGORDataset(dataset_root, split=area, 
                     train=training, pos_only=pos_only, 
                     transform=(transform_grd, transform_sat), 
                     ori_noise=ori_noise, use_osm_tiles=use_osm, 
                     use_50_n_osm_tiles=use_adapt, 
                     use_concat=use_concat)

loaded data pikle
osm tiles loaded for NewYork
InputData::__init__: load /scratch/izar/qngo/VIGOR/splits_new/NewYork/satellite_list.txt 23279
loaded data pikle
osm tiles loaded for Seattle
InputData::__init__: load /scratch/izar/qngo/VIGOR/splits_new/Seattle/satellite_list.txt 44055
loaded data pikle
osm tiles loaded for SanFrancisco
InputData::__init__: load /scratch/izar/qngo/VIGOR/splits_new/SanFrancisco/satellite_list.txt 68310
loaded data pikle
osm tiles loaded for Chicago
InputData::__init__: load /scratch/izar/qngo/VIGOR/splits_new/Chicago/satellite_list.txt 90618
Sat loaded, data size:90618
InputData::__init__: load  /scratch/izar/qngo/VIGOR/splits_new/NewYork/same_area_balanced_test.txt 13884
InputData::__init__: load  /scratch/izar/qngo/VIGOR/splits_new/Seattle/same_area_balanced_test.txt 25759
InputData::__init__: load  /scratch/izar/qngo/VIGOR/splits_new/SanFrancisco/same_area_balanced_test.txt 39866
InputData::__init__: load  /scratch/izar/qngo/VIGOR/splits_new/Chicago/sam

In [4]:
base_model_path = '/scratch/izar/qngo/models/VIGOR/'
os.listdir(base_model_path)

['samearea_HFoV360_samearea_lr_1e-04_osm_50n',
 'samearea_HFoV360_samearea_lr_1e-04test_rendered_tile',
 'samearea_HFoV360_samearea_lr_1e-04fusion_concat_image_osm_rendered_tile',
 'samearea_HFoV360_samearea_lr_1e-04normalized_osm_rendered_tile',
 'samearea_HFoV360_samearea_lr_1e-04_osm_rendered_tile',
 'samearea_HFoV360_samearea_lr_1e-04_osm']

In [5]:
selected_model = 'samearea_HFoV360_samearea_lr_1e-04normalized_osm_rendered_tile'
select_epoch = str(9)

test_model_path = os.path.join(base_model_path, selected_model, select_epoch, 'model.pt')
torch.cuda.empty_cache()
CVM_model = CVM(device, ori_noise, use_adapt=use_adapt, use_concat=False)
CVM_model.load_state_dict(torch.load(test_model_path))
CVM_model.to(device)
CVM_model.eval()

Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b0


/tmp/2472423/ipykernel_594231/4060473552.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  CVM_model.load_state_dict(torch.load(test_model_path))


CVM_VIGOR(
  (adapt_concat): Sequential(
    (0): Conv2d(6, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
  )
  (adapt_50_n): Sequential(
    (0): Conv2d(50, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
  )
  (grd_efficientnet): EfficientNet(
    (_conv_stem): Conv2dStaticCircularPadding(
      3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): Sequential(
        (0): custom_nn_CircularPad2d()
        (1): ZeroPad2d((0, 0, 0, 1))
      )
    )
    (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticCircularPadding(
          32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
          (static_padding): Sequential(
            (0): custom_nn_CircularPad2d()
            (1): ZeroPad2d((0, 0, 1, 1))
          )
        )
        (_bn1):

In [6]:
def run_infer(idx, dataset, model):
    grd, sat, gt, _, orientation, city, _ = dataset.__getitem__(idx)

    grd_feed = grd.unsqueeze(0)
    sat_feed = sat.unsqueeze(0)
    
    grd_feed = grd_feed.to(device)
    sat_feed = sat_feed.to(device)
    
    invTrans = transforms.Compose([ transforms.Normalize(mean = [ 0., 0., 0. ],
                                                     std = [ 1/0.229, 1/0.224, 1/0.225 ]),
                                transforms.Normalize(mean = [ -0.485, -0.456, -0.406 ],
                                                     std = [ 1., 1., 1. ]),
                               ])
    
    grd = invTrans(grd)
    sat = sat
    
    logits_flattened, heatmap, ori, matching_score_stacked, matching_score_stacked2, matching_score_stacked3, matching_score_stacked4, matching_score_stacked5, matching_score_stacked6 = model(grd_feed, sat_feed)
    matching_score_max1, _ = torch.max(matching_score_stacked, dim=1, keepdim=True)
    matching_score_max2, _ = torch.max(matching_score_stacked2, dim=1, keepdim=True)
    matching_score_max3, _ = torch.max(matching_score_stacked3, dim=1, keepdim=True)
    matching_score_max4, _ = torch.max(matching_score_stacked4, dim=1, keepdim=True)
    matching_score_max5, _ = torch.max(matching_score_stacked5, dim=1, keepdim=True)
    matching_score_max6, _ = torch.max(matching_score_stacked6, dim=1, keepdim=True)
    
    # grd = grd.cpu().detach().numpy() 
    # sat = sat.cpu().detach().numpy() 
    gt = gt.permute(1, 2, 0)
    gt = gt.cpu().detach().numpy() 
    loc_gt = np.unravel_index(gt.argmax(), gt.shape)
    
    
    orientation = orientation.permute(1, 2, 0).cpu().detach().numpy() 
    
    heatmap = torch.squeeze(heatmap, dim=0).permute(1, 2, 0)
    heatmap = heatmap.cpu().detach().numpy()
    loc_pred = np.unravel_index(heatmap.argmax(), heatmap.shape)
    ori = torch.squeeze(ori, dim=0).permute(1, 2, 0)
    ori = ori.cpu().detach().numpy()
    
    cos_pred_dense = ori[:, :, 0]
    sin_pred_dense = ori[:, :, 1]
    cos_pred, sin_pred = ori[loc_pred[0], loc_pred[1], :]
    
    
    cos_gt, sin_gt = orientation[loc_gt[0], loc_gt[1], :]
    a_acos_gt = math.acos(cos_gt)
    if sin_gt < 0:
        angle_gt = math.degrees(-a_acos_gt) % 360
    else: 
        angle_gt = math.degrees(a_acos_gt)

    return city, sat, grd, heatmap, loc_gt, loc_pred, sin_pred_dense, cos_pred_dense, sin_pred, cos_pred

In [7]:
def show_image(sat, grd, heatmap, loc_gt, loc_pred, sin_pred_dense, cos_pred_dense, sin_pred, cos_pred):
    plt.figure(figsize=(8,12))
    plt.imshow(  grd.permute(1, 2, 0)  )
    plt.axvline(grd.size()[2]/2, color='g')
    plt.axis('off')
    plt.savefig('figures/'+area+'_'+str(idx)+'_grd_'+'.png', bbox_inches='tight', pad_inches=0)
    
    plt.figure(figsize=(6,6))
    plt.imshow(  sat.permute(1, 2, 0)  )
    plt.imshow(heatmap,  norm=LogNorm(vmax=np.max(heatmap)), alpha=0.6, cmap='Reds')
    plt.scatter(loc_gt[1], loc_gt[0], s=300, marker='^', facecolor='g', label='GT', edgecolors='white')
    plt.scatter(loc_pred[1], loc_pred[0], s=300, marker='*', facecolor='gold', label='Ours', edgecolors='white')
    xx,yy = np.meshgrid(np.linspace(0,512,512),np.linspace(0,512,512))
    cos_angle = ori[:,:,0]
    sin_angle = ori[:,:,1]
    plt.quiver(xx[::40, ::40], yy[::40, ::40], -sin_pred_dense[::40, ::40], cos_pred_dense[::40, ::40], linewidths=0.2, scale=14, width=0.01) # plot the predicted rotation angle + 90 degrees
    plt.quiver(loc_pred[1], loc_pred[0], -sin_pred, cos_pred, color='gold', linewidths=0.2, scale=10, width=0.015)
    plt.quiver(loc_gt[1], loc_gt[0], -np.sin(angle_gt / 180 * np.pi), np.cos(angle_gt / 180 * np.pi), color='g', linewidths=0.2, scale=10, width=0.015)
    plt.axis('off')
    plt.legend(loc=2, framealpha=0.8, labelcolor='black', prop={'size': 15})

In [8]:
len(vigor)

52605

In [9]:
from tqdm import tqdm

distance_array = np.zeros(len(vigor))

for i in tqdm(range(len(vigor))):
    idx = i
    city, sat, grd, heatmap, loc_gt, loc_pred, sin_pred_dense, cos_pred_dense, sin_pred, cos_pred = run_infer(idx, vigor, CVM_model)
    get_meter_distance(loc_pred, loc_gt, city, 0)
    distance_array[i] = get_meter_distance(loc_pred, loc_gt, city, 0)

show_image(sat, grd, heatmap, loc_gt, loc_pred, sin_pred_dense, cos_pred_dense, sin_pred, cos_pred)
# plt.savefig('figures/'+area+'_'+str(idx)+'_noise_in_orientation_'+str(ori_noise)+'.png', bbox_inches='tight', pad_inches=0)
# print('Images are written to figures/')
save_qual = os.path.join('/scratch/izar/qngo', 'qualitative', 'distance_test.npy')
with open(save_qual, 'wb') as f:
    np.save(f, distance_array)

  0%|▎                                                                                                                                                                          | 112/52605 [00:14<1:56:58,  7.48it/s]


KeyboardInterrupt: 

In [10]:
distance_array[100]

np.float64(4.3883600000000005)

In [11]:
save_qual = os.path.join('/scratch/izar/qngo', 'qualitative', 'distance_test.npy')
with open(save_qual, 'wb') as f:
    np.save(f, distance_array)